# [문제]
- ChatPromptTemplate() : 역할 부여
- InMemoryChatMessageHistory()
- RunnableWithMessageHistory()
- chain 구성

## [문제] PromptTemplate 내 변수 설정: Chain

template = '''
당신은 이력서 작성 컨설턴트입니다.
아래 정보를 바탕으로 지원자 입장에서 2000자 이내로 이력서를 작성합니다.
문장은 자연스럽고 매끄럽게 작성합니다.

이름: {name}
경력: {experience}
기술: {skills}
'''

In [ ]:
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.chat_history import InMemoryChatMessageHistory
from langchain_openai import ChatOpenAI
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from dotenv import load_dotenv


load_dotenv()

llm = ChatOpenAI(
    model='gpt-4.1-nano',
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()],
    max_completion_tokens=150,
)

messages = [
    ('system', '''당신은 이력서 작성 컨설턴트입니다.
아래 정보를 바탕으로 지원자 입장에서 2000자 이내로 이력서를 작성합니다.
문장은 자연스럽고 매끄럽게 작성합니다.'''),
	('placeholder', '{chat_history}'),
	# ('user', '이름: {name}\n경력: {experience}\n기술: {skills}')
	('user', '{query}')
]

prompt = ChatPromptTemplate.from_messages(messages=messages)

In [4]:
history = InMemoryChatMessageHistory()

def get_history():
    return history

chain = prompt | llm

with_message_history = RunnableWithMessageHistory(
    chain,
    get_history,
    input_messages_key='query',
    history_messages_key='chat_history'
)

ai_message = with_message_history.invoke({
    'query': '''name: 박보검\nexperience: 웹 개발 신입\nskills: Python, HTML, CSS, LLM''' 
})

박보검  
연락처: [전화번호] | 이메일: [이메일 주소] | 주소: [주소]  

목표  
열정과 성실함을 바탕으로 웹 개발 분야에서 성장하고자 하는 신입 개발자입니다. Python, HTML, CSS, 그리고 최신 인공지능 기술인 LLM(대형 언어 모델)에 대한 이해를 바탕으로 기업의 디지털 전환과 서비스 개선에 기여하고 싶습니다. 빠른 학습 능력과 팀워크를 통해 주어진 업무를 성실히 수행하며, 지속적인 자기개발을 지향합니다.

학력  
[대학교명], [전공] – [졸업 예정 또는 졸업

In [5]:
history

InMemoryChatMessageHistory(messages=[HumanMessage(content='name: 박보검\nexperience: 웹 개발 신입\nskills: Python, HTML, CSS, LLM', additional_kwargs={}, response_metadata={}), AIMessage(content='박보검  \n연락처: [전화번호] | 이메일: [이메일 주소] | 주소: [주소]  \n\n목표  \n열정과 성실함을 바탕으로 웹 개발 분야에서 성장하고자 하는 신입 개발자입니다. Python, HTML, CSS, 그리고 최신 인공지능 기술인 LLM(대형 언어 모델)에 대한 이해를 바탕으로 기업의 디지털 전환과 서비스 개선에 기여하고 싶습니다. 빠른 학습 능력과 팀워크를 통해 주어진 업무를 성실히 수행하며, 지속적인 자기개발을 지향합니다.\n\n학력  \n[대학교명], [전공] – [졸업 예정 또는 졸업', additional_kwargs={}, response_metadata={'finish_reason': 'length', 'model_name': 'gpt-4.1-nano-2025-04-14', 'system_fingerprint': 'fp_eede8f0d45'}, id='run-e24e09ab-6130-48f0-954f-806b65dfc3d6-0')])

In [6]:
ai_message_02 = with_message_history.invoke({'query': '내 이름이 뭐야?'}) 

당신의 이름은 박보검입니다.

In [7]:
history

InMemoryChatMessageHistory(messages=[HumanMessage(content='name: 박보검\nexperience: 웹 개발 신입\nskills: Python, HTML, CSS, LLM', additional_kwargs={}, response_metadata={}), AIMessage(content='박보검  \n연락처: [전화번호] | 이메일: [이메일 주소] | 주소: [주소]  \n\n목표  \n열정과 성실함을 바탕으로 웹 개발 분야에서 성장하고자 하는 신입 개발자입니다. Python, HTML, CSS, 그리고 최신 인공지능 기술인 LLM(대형 언어 모델)에 대한 이해를 바탕으로 기업의 디지털 전환과 서비스 개선에 기여하고 싶습니다. 빠른 학습 능력과 팀워크를 통해 주어진 업무를 성실히 수행하며, 지속적인 자기개발을 지향합니다.\n\n학력  \n[대학교명], [전공] – [졸업 예정 또는 졸업', additional_kwargs={}, response_metadata={'finish_reason': 'length', 'model_name': 'gpt-4.1-nano-2025-04-14', 'system_fingerprint': 'fp_eede8f0d45'}, id='run-e24e09ab-6130-48f0-954f-806b65dfc3d6-0'), HumanMessage(content='내 이름이 뭐야?', additional_kwargs={}, response_metadata={}), AIMessage(content='당신의 이름은 박보검입니다.', additional_kwargs={}, response_metadata={'finish_reason': 'stop', 'model_name': 'gpt-4.1-nano-2025-04-14', 'system_fingerprint': 'fp_eede8f0d45'}, id='run-bbdab1de-3e5b-47f6-8cc2-2a20fd4a7898-0')])